In [155]:
# Часть 1.1:
# Ответьте на вопросы:
# В одной из компаний дата-аналитик Олег нашел фичу, которая отсевает 100% фродовый сценарий из группы 2 человека в день.
# Как считаете, необходимо ли реализовывать это на бою? Как быстро? Объясните почему?

# # Думаю, что реализация данной фичи не целесообразна или как минимум не приоритетна. Маленькая группа и, как следствие,
# # большой срок окупаемости процесса внедрения данной фичи.



# В компании уже много лет реализован на бою один из фродовых сценариев. Произошел сбой и клиентов по данному сценарию стали пропускать.
# Неисправность оперативно починили, но скопилось достаточно информации для анализа.
# Дата-аналитик Олег провел исследование и выяснил, что клиенты, прошедшие в сервис, не так уж и плохи.
# Они гораздо лучше тех, кого обычно закрывают правилами антифрода. Как считаете, что стоит предпринять в данной ситуации? Объясните почему?

# # Пересмотреть сценарий и возможно изменить условия пропуска клиентов по нему. Перед реализацией на бою обновленного фродового сценария
# # провести А/В тестирование с целью минимизации потерь в случае ошибочности гипотезы.



# Дата-аналитик Олег хочет рассчитать процент дефолта при котором клиентам выгодно выдавать ПЕРВЫЙ займ.
# Известно, что в среднем клиенты берут еще три займа после первого и приносят 20% прибыли на каждый займ. Помогите Олегу рассчитать этот процент.
# Как считаете, условие из п.3 возможно применить как правило антифрода в реальной жизни? Объясните почему?

# # Для того чтобы не уйти в минус после первого займа процент дефолта должен быть не выше (1 - 5/6), или 17,778 %.
# # Люди, не вернувшие первый займ, не придут за вторым и все последующие займы будут приносить прибыль.
# # Данное правило в реальной жизни, думаю, не применимо, так как задача антифрод команды не допустить потенциальных фродеров
# # даже до первого займа.

In [155]:
# Часть 1.2:
# Придумайте три правила антифрода, не указанных в презентации, которые бы Вы применили при выдаче займов.

# 1. клиент запрашивает сумму больше чем 10% клиентов не имея автомобиля и недвижимости.
# в случае невозможности оплаты (например потеря работы) будет "нечего взять"

# 2. запрашивают гораздо больше своего дохода. половина дохода меньше платежа.
# не планирует возвращать.

# 3. снятие большой суммы (х10 от среднего) через банкомат при возрасте 60+. повод запросить дополнительную информацию.
# подозрение на "социальную инженерию"

In [156]:
import json
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import random
import warnings
warnings.filterwarnings('ignore')
from dataclasses import asdict
from datetime import datetime, date
import io
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/shiftlogs/application_train.csv')

df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
# Часть 2:
# Реализуйте три правила антифрода у себя в сервисе.

df['AMT_CREDIT'].describe(percentiles = [.1, .9])

# 10% клиентов запрашивают более 1_135_000

count    3.075110e+05
mean     5.990260e+05
std      4.024908e+05
min      4.500000e+04
10%      1.800000e+05
50%      5.135310e+05
90%      1.133748e+06
max      4.050000e+06
Name: AMT_CREDIT, dtype: float64

In [158]:
froad_df = pd.DataFrame()
froad_df['SK_ID_CURR'] = df['SK_ID_CURR']

def alert(df):
    if df['AMT_CREDIT'] >= 1_135_000 and (df['FLAG_OWN_CAR'] == 'N') and (df['FLAG_OWN_REALTY'] == 'N'):
        return 1
    else:
        return 0
    
froad_df['froad_flag_1'] = df.apply(alert, axis=1)

froad_df['froad_flag_1'].value_counts()

0    301989
1      5522
Name: froad_flag_1, dtype: int64

In [159]:
froad_df['froad_flag_2'] = np.where(df['AMT_INCOME_TOTAL']/2 < df['AMT_ANNUITY'], 1, 0)

froad_df['froad_flag_2'].value_counts()

0    304981
1      2530
Name: froad_flag_2, dtype: int64

In [160]:
df = pd.read_csv('/kaggle/input/shiftlogs/credit_card_balance.csv')

df = df[['SK_ID_CURR', 'AMT_DRAWINGS_ATM_CURRENT']]

df

,SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT
0,378907,0.0
1,363914,2250.0
2,371185,0.0
3,337855,2250.0
4,126868,0.0
...,...,...
3840307,328243,NaN
3840308,347207,0.0
3840309,215757,270000.0
3840310,430337,NaN


In [161]:
df['mean'] = df['AMT_DRAWINGS_ATM_CURRENT'].groupby(df['SK_ID_CURR']).mean()
# df.fillna(value=0, inplace=True)

df

,SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT,mean
0,378907,0.0,NaN
1,363914,2250.0,NaN
2,371185,0.0,NaN
3,337855,2250.0,NaN
4,126868,0.0,NaN
...,...,...,...
3840307,328243,NaN,NaN
3840308,347207,0.0,NaN
3840309,215757,270000.0,NaN
3840310,430337,NaN,NaN


In [162]:
df['froad_flag_3'] = np.where(df['AMT_DRAWINGS_ATM_CURRENT'] * 10 > df['mean'], 1, 0)

df

,SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT,mean,froad_flag_3
0,378907,0.0,NaN,0
1,363914,2250.0,NaN,0
2,371185,0.0,NaN,0
3,337855,2250.0,NaN,0
4,126868,0.0,NaN,0
...,...,...,...,...
3840307,328243,NaN,NaN,0
3840308,347207,0.0,NaN,0
3840309,215757,270000.0,NaN,0
3840310,430337,NaN,NaN,0


In [163]:
df['froad_flag_3'].value_counts()

0    3832883
1       7429
Name: froad_flag_3, dtype: int64